In [1]:
import os
import pandas as pd
import glob

In [2]:
pollutants = ['NO2', 'O3','PM10','PM25']
fnames = []
for p in pollutants:
    directory_mask = os.path.join("../../data/9. data_for_forecasting/pollutant", f"{p}*")
    fnames.append(glob.glob(directory_mask))

In [3]:
def computeDailyAvg(fname):
    df = pd.read_csv(fname, skiprows=1, header=None, na_values= ' ', index_col=0)
    df.drop(columns=df.columns[-1],inplace=True)
    df.index = pd.to_datetime(df.index)
    df_avg = df.groupby(df.index.day).mean(numeric_only=True)
    daylist = []
    for day in df_avg.index:
        day = str(df.index.year[0])+'-'+str(df.index.month[0])+'-'+str(day)
        daylist.append(day)
    df_avg.index = daylist
    df_avg.index = pd.to_datetime(df_avg.index)
    df_avg.index.name = 'DATE'
    return df_avg

In [4]:
def mergeMonthlyData(pollutant, files):
    df_combined = None
    for f in files:
        idf = computeDailyAvg(f)
        colsToDrop = idf.columns
        df_combined = pd.concat([df_combined, idf], axis=0)
        df_combined[pollutant] = df_combined.mean(axis=1)
        df_combined.drop(columns=colsToDrop, inplace=True)
    return df_combined
    

In [5]:
master_df = None
for i, p in enumerate(pollutants):
    idf = mergeMonthlyData(p, fnames[i])
    if master_df is None:
        master_df = idf
        continue
    master_df = master_df.merge(idf, on='DATE', how='inner')

In [6]:
#Add powiat voivodeship data
master_df['powiat_voivod'] = 'powiat warszawa, mazowieckie'

In [7]:
master_df.sort_index(inplace=True)

In [8]:
master_df.to_csv("../../data/9. data_for_forecasting/Combined_pollutant_data_forecasting.csv")